In [ ]:
import pandas as pd
#import csv
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import seaborn as sb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install nltk

In [ ]:
!pip install seaborn

In [ ]:
train_news = pd.read_csv("/content/drive/My Drive/Fake News Detection/Fact Checking/Model/train.csv")
test_news = pd.read_csv("/content/drive/My Drive/Fake News Detection/Fact Checking/Model/test.csv")
valid_news = pd.read_csv("/content/drive/My Drive/Fake News Detection/Fact Checking/Model/valid.csv")

In [ ]:
valid_news.tail()
print("train_news data below : ")
train_news.head()

train_news data below : 


,Statement,Label
0,Says the Annies List political group supports ...,False
1,When did the decline of coal start? It started...,True
2,"Hillary Clinton agrees with John McCain ""by vo...",True
3,Health care reform legislation is likely to ma...,False
4,The economic turnaround started at the end of ...,True


In [ ]:
#code to find out the missing values in the training set
train_news.isnull().sum()
#seems like there is no missing data in the training set

Statement    0
Label        0
dtype: int64

In [ ]:
#code to find out the missing values in the test test and the validation set
valid_news.isnull().sum()
test_news.isnull().sum()

#turns out there are 2 missing values in the validation set and 0 in test set

Statement    0
Label        0
dtype: int64

In [ ]:
#checking out the data types of the columns
test_news.dtypes

Statement    object
Label          bool
dtype: object

In [ ]:
#here convert the Label values "True" = 1 and "False" = 0
#because the machine understands numbers better than words

train_news.Label = train_news.Label.astype(int)
test_news.Label = test_news.Label.astype(int)

test_news.tail()

,Statement,Label
2546,"For the first time in more than a decade, impo...",1
2547,Says Donald Trump has bankrupted his companies...,1
2548,"John McCain and George Bush have ""absolutely n...",1
2549,A new poll shows 62 percent support the presid...,0
2550,No one claims the report vindicating New Jerse...,0


In [ ]:
#put statement into x axis, and
#put labels into y axis

train_news_x = train_news["Statement"]
train_news_y = train_news["Label"]

test_news_x = test_news["Statement"]
test_news_y = test_news["Label"]
test_news_x.head()
#train_news_y.head()

0    Building a wall on the U.S.-Mexico border will...
1    Wisconsin is on pace to double the number of l...
2    Says John McCain has done nothing to help the ...
3    Suzanne Bonamici supports a plan that will cut...
4    When asked by a reporter whether hes at the ce...
Name: Statement, dtype: object

In [ ]:
cv = CountVectorizer()

In [ ]:
train_news_cv = cv.fit_transform(train_news_x)
array = train_news_cv.toarray()
array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Here we are going to feed this into our ML model (Naive Bayes Classifier) 


In [ ]:
cv.inverse_transform(array[0])

[array(['abortions', 'annies', 'demand', 'group', 'list', 'on',
        'political', 'says', 'supports', 'the', 'third', 'trimester'],
       dtype='<U23')]

In [ ]:
#here we will be using "Tf-Idf" and do the same thing as we did with CountVectorizer
tf_idf_vec = TfidfVectorizer(min_df= 1, stop_words= 'english')

In [ ]:
train_news_tfidf = tf_idf_vec.fit_transform(train_news_x)
array = train_news_tfidf.toarray()
array

'''test_news_tfidf = tf_idf_vec.fit_transform(test_news_x)
array = test_news_tfidf.toarray()
array '''

#you cant see any visible change because the corpus is really very big

'test_news_tfidf = tf_idf_vec.fit_transform(test_news_x)\narray = test_news_tfidf.toarray()\narray '

CountVectorizer = just counts the words
tf_idf = takes those words which are not repetitive or are unique, so that word would mean something as opposed to trivial words like 'a', 'the' etc. You get the idea

In [ ]:
mnb = MultinomialNB()

In [ ]:
mnb.fit(train_news_tfidf, train_news_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Now we will predict on test set

In [ ]:
test_news_tfidf = tf_idf_vec.transform(test_news)

In [ ]:
pred = mnb.predict(train_news_tfidf)

In [ ]:
pred

array([0, 1, 1, ..., 1, 0, 0])

In [ ]:
actual = np.array(train_news_y)

In [ ]:
actual

array([0, 1, 1, ..., 1, 0, 0])

In [ ]:
len(pred)

10240

In [ ]:
count = 0

In [ ]:

for i in range (10240):
  if pred[i] == actual[i]:
    count = count + 1

In [ ]:
count

7952

In [ ]:
accuracy = (7952/10240)*100

In [ ]:
accuracy

77.65625